In [1]:
import platform

In [2]:
platform.architecture()

('64bit', 'WindowsPE')

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decimal import Decimal
import math
from sklearn.preprocessing import LabelEncoder
%matplotlib inline 

In [4]:
gs=pd.read_csv('gender_submission.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [5]:
gs.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [6]:
print(len(train.columns))
print(len(test.columns))

12
11


In [7]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [9]:
print(len(train))
print(len(test))

891
418


**change missing value**

In [10]:
print('train')
for col in train.columns:
    if train[col].isnull().sum()>0:
        print(col, train[col].isnull().sum())
        
print('test')
for col in test.columns:
    if test[col].isnull().sum()>0:
        print(col, test[col].isnull().sum())

train
Age 177
Cabin 687
Embarked 2
test
Age 86
Fare 1
Cabin 327


In [11]:
print(len(train.Age.unique()))
print(len(test.Age.unique()))

89
80


In [10]:
train.Age.describe()


count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [11]:
test.Age.describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [12]:
def check_int(data, columns):
    lst=[]
    for col in columns:
        for idx in data.index:
            frac,intn=math.modf(data.loc[idx,col])
            if frac!=0.0:
                lst.append(data.loc[idx,col])
    if lst==[]:
        print('all int')
    else:
        print(lst)

In [13]:
check_int(test,['Age'])

[34.5, nan, nan, 22.5, nan, nan, 18.5, nan, nan, nan, nan, nan, nan, nan, 28.5, 32.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 18.5, nan, 18.5, nan, nan, nan, nan, nan, nan, 26.5, nan, nan, nan, 60.5, nan, nan, nan, nan, nan, nan, nan, nan, 11.5, nan, nan, 0.33, nan, nan, nan, nan, nan, nan, 40.5, nan, nan, nan, nan, 0.92, 32.5, nan, nan, 36.5, nan, nan, nan, nan, nan, nan, nan, 0.75, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.83, nan, nan, nan, nan, nan, 0.17, nan, nan, 14.5, nan, nan, nan, nan, nan, nan, nan, nan, 38.5, nan, nan]


In [14]:
check_int(train,['Age'])

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 28.5, nan, nan, nan, nan, 0.83, nan, nan, nan, nan, nan, nan, 14.5, 70.5, nan, 32.5, 32.5, nan, nan, nan, 36.5, 55.5, 40.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 45.5, nan, nan, 20.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 23.5, nan, nan, nan, nan, nan, 0.92, nan, nan, nan, 45.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.75, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 40.5, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.75, nan, nan, nan, nan, nan, nan, nan, 24.5, nan, nan, nan, nan, nan, nan, nan, nan, 28.5, nan, nan, nan, 0.67, nan, nan, 30.5, nan, nan, nan, nan, nan, nan, nan, nan, 0.42, 30.5, nan, nan, nan, nan, 0.83, nan, nan, nan, 34.5,

Deal with missinve value

In [15]:
train=train[train.Embarked.notnull()]
train['Cabin']=train.Cabin.fillna(value='X')
train['CabinType']=[i[0] for i in train.Cabin]
train=train.drop(['Cabin'],axis=1)
train['Age']=train.Age.fillna(value=0.0)
train['isAge']=[int(i!=0.0) for i in train.Age]
train['Age']=train.Age.astype(int)

test=test[test.Fare.notnull()]
test['Cabin']=test.Cabin.fillna(value='X')
test['CabinType']=[i[0] for i in test.Cabin]
test=test.drop(['Cabin'],axis=1)
test['Age']=test.Age.fillna(value=0.0)
test['isAge']=[int(i!=0.0) for i in test.Age]
test['Age']=test.Age.astype(int)


In [16]:
#recheck:
print('train')
for col in train.columns:
    if train[col].isnull().sum()>0:
        print(col, train[col].isnull().sum())
        
print('test')
for col in test.columns:
    if test[col].isnull().sum()>0:
        print(col, test[col].isnull().sum())

train
test


In [17]:
train.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinType,isAge
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,S,X,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,S,X,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,S,C,1
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,S,X,1
5,6,0,3,"Moran, Mr. James",male,0,0,0,330877,8.4583,Q,X,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,S,E,1
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.0750,S,X,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,S,X,1
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,C,X,1


Deal with Ticket 

In [18]:
print(len(train.Ticket.unique()))
print(len(test.Ticket.unique()))

680
362


In [19]:
for idx in train.index:
    check=train.loc[idx,'Ticket'].split(' ')
    if len(check)>2:
        print(check)

['STON/O', '2.', '3101294']
['STON/O', '2.', '3101280']
['STON/O', '2.', '3101275']
['STON/O', '2.', '3101293']
['STON/O', '2.', '3101289']
['STON/O', '2.', '3101269']
['STON/O', '2.', '3101274']
['SC/AH', 'Basle', '541']
['STON/O', '2.', '3101286']
['STON/O', '2.', '3101273']
['STON/O', '2.', '3101292']
['STON/O', '2.', '3101285']
['STON/O', '2.', '3101288']


In [20]:
for idx in train.index:
    check=train.loc[idx,'Ticket'].split(' ')
    if len(check)==1:
        train.loc[idx,'T_type']='number'
    elif len(check)==2:
        train.loc[idx,'T_type']=check[0]
    else:
        train.loc[idx,'T_type']=' '.join(check[0:-1])
        
for idx in test.index:
    check=test.loc[idx,'Ticket'].split(' ')
    if len(check)==1:
        test.loc[idx,'T_type']='number'
    elif len(check)==2:
        test.loc[idx,'T_type']=check[0]
    else:
        test.loc[idx,'T_type']=' '.join(check[0:-1])

In [21]:
test.T_type.unique()

array(['number', 'A/4', 'W.E.P.', 'SC/PARIS', 'STON/O2.', 'PC', 'C',
       'A/5.', 'SC/AH', 'C.A.', 'W./C.', 'SOTON/O.Q.', 'STON/O 2.',
       'SC/A.3', 'F.C.C.', 'F.C.', 'A./5.', 'PP', 'STON/OQ.', 'SOTON/OQ',
       'CA', 'SC/A4', 'S.O./P.P.', 'CA.', 'S.O.C.', 'SOTON/O2', 'AQ/4',
       'A. 2.', 'SC', 'A/5', 'SC/Paris', 'LP', 'AQ/3.', 'S.C./PARIS',
       'A.5.'], dtype=object)

In [22]:
train.T_type.unique()

array(['A/5', 'PC', 'STON/O2.', 'number', 'PP', 'A/5.', 'C.A.', 'A./5.',
       'SC/Paris', 'S.C./A.4.', 'A/4.', 'CA', 'S.P.', 'S.O.C.', 'SO/C',
       'W./C.', 'SOTON/OQ', 'W.E.P.', 'STON/O 2.', 'A4.', 'C',
       'SOTON/O.Q.', 'SC/PARIS', 'S.O.P.', 'A.5.', 'Fa', 'CA.', 'F.C.C.',
       'W/C', 'SW/PP', 'SCO/W', 'P/PP', 'SC', 'SC/AH', 'A/S',
       'SC/AH Basle', 'A/4', 'WE/P', 'S.W./PP', 'S.O./P.P.', 'F.C.',
       'SOTON/O2', 'S.C./PARIS', 'C.A./SOTON'], dtype=object)

In [23]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinType,isAge,T_type
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,S,X,1,A/5
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C,C,1,PC
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,S,X,1,STON/O2.
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,S,C,1,number
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,S,X,1,number


Deal with name

In [27]:
train['Name_Length']=[len(N) for N in train.Name]
train['Last_Name']=[N.split(',')[0] for N in train.Name]

test['Name_Length']=[len(N)for N in test.Name]
test['Last_Name']=[N.split(',')[0] for N in test.Name]


In [28]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinType,isAge,T_type,Name_Length,Last_Name
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,S,X,1,A/5,23,Braund
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C,C,1,PC,51,Cumings
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,S,X,1,STON/O2.,22,Heikkinen
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,S,C,1,number,44,Futrelle
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,S,X,1,number,24,Allen


In [29]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,CabinType,isAge,T_type,Name_Length,Last_Name
0,892,3,"Kelly, Mr. James",male,34,0,0,330911,7.8292,Q,X,1,number,16,Kelly
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,S,X,1,number,32,Wilkes
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,Q,X,1,number,25,Myles
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,S,X,1,number,16,Wirz
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,S,X,1,number,44,Hirvonen


In [43]:
# drop duplicate
train=train.drop_duplicates()
print(len(train))
test=test.drop_duplicates()
print(len(test))

889
417


In [44]:
#Check unique value
print('train')
for col in train.columns:
    print(col, len(train[col].unique()))
print('\n\ntest')
for col in test.columns:
    print(col, len(test[col].unique()))

train
PassengerId 889
Survived 2
Pclass 3
Name 889
Sex 2
Age 71
SibSp 7
Parch 7
Ticket 680
Fare 247
Embarked 3
CabinType 9
isAge 2
T_type 44
Name_Length 50
Last_Name 665


test
PassengerId 417
Pclass 3
Name 417
Sex 2
Age 64
SibSp 7
Parch 8
Ticket 362
Fare 169
Embarked 3
CabinType 8
isAge 2
T_type 35
Name_Length 46
Last_Name 351


In [49]:
# label encoding
le=LabelEncoder()
le.fit(train.PassengerId)
train['Pid']=le.transform(train.PassengerId)
le.fit(train.Ticket)
train['Tkt']=le.transform(train.Ticket)
le.fit(train.Last_Name)
train['Family']=le.transform(train.Last_Name)

le=LabelEncoder()
le.fit(test.PassengerId)
test['Pid']=le.transform(test.PassengerId)
le.fit(test.Ticket)
test['Tkt']=le.transform(test.Ticket)
le.fit(test.Last_Name)
test['Family']=le.transform(test.Last_Name)


In [52]:
train_ready=train.drop(labels=['PassengerId','Name','Ticket','Last_Name'],axis=1)
test_ready=test.drop(labels=['PassengerId','Name','Ticket','Last_Name'],axis=1)


In [57]:
def one_hot(data,columns=[]):
    for col in columns:
        unilst=data[col].unique()
        for val in unilst:
            data[col+'_'+str(val)]=[int(i==val) for i in data[col]]
        data=data.drop([col],axis=1)
    return data
            

In [71]:
train_ready=one_hot(train_ready,['Pclass','Sex','SibSp','Parch','Embarked','CabinType','T_type'])
test_ready=one_hot(test_ready,['Pclass','Sex','SibSp','Parch','Embarked','CabinType','T_type'])

In [74]:
train_ready.head()

,Survived,Age,Fare,isAge,Name_Length,Pid,Tkt,Family,Pclass_3,Pclass_1,...,T_type_A/S,T_type_SC/AH Basle,T_type_A/4,T_type_WE/P,T_type_S.W./PP,T_type_S.O./P.P.,T_type_F.C.,T_type_SOTON/O2,T_type_S.C./PARIS,T_type_C.A./SOTON
0,0,22,7.2500,1,23,0,522,73,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,38,71.2833,1,51,1,595,136,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,26,7.9250,1,22,2,668,251,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,35,53.1000,1,44,3,48,198,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,35,8.0500,1,24,4,471,11,1,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
test_ready.head()

,Age,Fare,isAge,Name_Length,Pid,Tkt,Family,Pclass_3,Pclass_2,Pclass_1,...,T_type_SOTON/O2,T_type_AQ/4,T_type_A. 2.,T_type_SC,T_type_A/5,T_type_SC/Paris,T_type_LP,T_type_AQ/3.,T_type_S.C./PARIS,T_type_A.5.
0,34,7.8292,1,16,0,152,172,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,47,7.0000,1,32,1,221,338,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,9.6875,1,25,2,73,231,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,27,8.6625,1,16,3,147,343,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22,12.2875,1,44,4,138,150,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
X_train=train_ready.drop('Survived',axis=1)
y_train=train['Survived']
X_test=test_ready

## Benchmakr Modeling

In [154]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegressionCV as lrcv
from sklearn.linear_model import LogisticRegression as lr
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import GradientBoostingClassifier as gbc
from xgboost import XGBClassifier as xgbc
from lightgbm import LGBMClassifier as lgbmc


In [88]:
LRCV=lrcv(Cs=[10**i for i in range(2,5)],cv=5,scoring='roc_auc',solver='liblinear',penalty='l1',class_weight='balanced',n_jobs=6,random_state=1001)
LRCV.fit(X_train,y_train)
print('score for Cs', LRCV.scores_[1].mean(axis=0))
print('Best Score', LRCV.scores_[1].mean(axis=0).max())
lst=list(LRCV.scores_[1].mean(axis=0))
index=lst.index(LRCV.scores_[1].mean(axis=0).max())
print('All chosen Cs',LRCV.Cs_)
print('C for Best Score',LRCV.Cs_[index])

score for Cs [0.84683879 0.83826326 0.83654737]
Best Score 0.8468387872246481
All chosen Cs [  100  1000 10000]
C for Best Score 100


## feature selection

In [95]:
LR=lr(C=100,penalty='l1',solver='liblinear',class_weight='balanced',n_jobs=6,random_state=1001)
LR.fit(X_train,y_train)

c:\users\wuyuhang\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:1232: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 6.
  " = {}.".format(self.n_jobs))


LogisticRegression(C=100, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=6, penalty='l1', random_state=1001,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [101]:
print(len(LR.coef_[0]))
print(LR.coef_[0])

82
[-3.98322880e-02  6.83167279e-03  1.14436390e+00  4.60787468e-02
 -1.92219966e-05 -3.22842704e-03 -5.38883993e-04 -3.01025460e-01
  5.99877313e-02  5.35090524e-02 -1.37977793e+00  1.19004138e+00
  3.87230494e-01  4.52423096e-01 -1.80842923e+00 -1.43521264e+00
  6.63486417e-01  0.00000000e+00 -9.27455642e+00  1.03055239e-01
  2.13750012e-01  0.00000000e+00 -2.12726501e+00  9.98751114e-01
 -6.17982772e+00  0.00000000e+00 -2.52788040e-01  1.97265103e-01
  4.59634384e-01 -6.19655310e-01 -2.34555159e-01  1.99303049e+00
 -2.93809877e+00  1.18088330e+00  4.86065747e-01  2.01200889e-01
  9.66302742e-02 -4.17445153e+00  9.37117716e-01  9.31174697e-01
  9.71841469e-01  8.85038593e-02  2.87767772e+00 -1.05732653e-01
  1.60359051e+00 -2.54520741e+00 -1.75227365e-01 -1.27759863e+00
 -3.10105405e+00 -5.37030305e+00 -1.85247481e+00 -4.17503238e+00
  4.72759346e+00 -1.25646299e+00  1.79587044e+00 -4.02356807e+00
  3.48864540e+00 -1.67514760e+00  2.11415262e+00  1.99310778e-01
  1.83638187e+00 -1.10

In [112]:
param_grid={
    'n_estimators': [1000],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [2],
    'max_features':['sqrt'],
    'oob_score': [True]
}

RF=rf(
    n_estimators=1000,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    oob_score=True,
    class_weight='balanced_subsample',
    n_jobs=3,
    random_state=1001
)

model=GridSearchCV(estimator=RF, param_grid=param_grid, cv=5, n_jobs=2, scoring='roc_auc')
model.fit(X_train,y_train)
print(model.best_params_)
print(model.best_score_)

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 1000, 'oob_score': True}
0.8725128082926247


In [115]:
RF=rf(
    n_estimators=1000,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    oob_score=True,
    class_weight='balanced_subsample',
    n_jobs=3,
    random_state=1001
)

RF.fit(X_train,y_train)
print(RF.feature_importances_)

[5.30727010e-02 8.24185136e-02 7.12444297e-03 8.07831867e-02
 5.03705375e-02 8.75612448e-02 5.36263576e-02 4.79057667e-02
 2.36175897e-02 1.17326485e-02 1.61533550e-01 1.66936639e-01
 1.08170557e-02 9.02156264e-03 3.95371126e-03 2.16781208e-03
 1.57963675e-03 4.62502233e-04 1.46061582e-03 1.42615980e-02
 9.18038462e-03 5.81504281e-03 7.23970776e-04 1.62897011e-04
 5.62438017e-04 0.00000000e+00 1.23346662e-02 1.07195719e-02
 4.91513250e-03 3.37353713e-02 4.22211344e-03 7.26278659e-03
 3.96481254e-04 4.53281455e-03 1.83453801e-03 3.61546202e-03
 1.32009651e-03 0.00000000e+00 4.63731973e-04 5.04205524e-03
 7.36200240e-04 6.69550540e-03 1.56284052e-04 1.15307452e-03
 2.43288706e-03 2.82540804e-05 1.13682398e-04 0.00000000e+00
 4.53935748e-05 9.15281949e-04 0.00000000e+00 7.03444703e-04
 0.00000000e+00 2.23798567e-03 4.61534744e-04 0.00000000e+00
 4.10247453e-03 0.00000000e+00 6.36415139e-04 2.80270994e-04
 4.26885252e-04 0.00000000e+00 4.93999293e-05 0.00000000e+00
 7.92455757e-04 4.750570

In [121]:
df=pd.DataFrame({'Feature':X_train.columns, 'Importance':RF.feature_importances_})
df.sort_values(by='Importance',ascending=False)
feature_remove=list(df[df.Importance<0.005].Feature)

In [122]:
selection=[i for i in list(X_train.columns) if i not in feature_remove]
X_train_ready=X_train[selection]
y=y_train
X_test_ready=X_test[selection]

In [126]:
print(len(X_train_ready.columns))
print(len(X_test_ready.columns))

23
23


## modeling

In [125]:
# LR

59

In [128]:
LRCV=lrcv(Cs=[10**i for i in range(-5,5)],cv=5,scoring='roc_auc',solver='liblinear',penalty='l1',class_weight='balanced',n_jobs=6,random_state=1001)
LRCV.fit(X_train_ready,y)
print('score for Cs', LRCV.scores_[1].mean(axis=0))
print('Best Score', LRCV.scores_[1].mean(axis=0).max())
lst=list(LRCV.scores_[1].mean(axis=0))
index=lst.index(LRCV.scores_[1].mean(axis=0).max())
print('All chosen Cs',LRCV.Cs_)
print('C for Best Score',LRCV.Cs_[index])

score for Cs [0.5        0.59709084 0.68191581 0.72886155 0.84194697 0.85369573
 0.85364838 0.85343276 0.85351322 0.85353996]
Best Score 0.8536957268311829
All chosen Cs [1.e-05 1.e-04 1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03 1.e+04]
C for Best Score 1.0


In [129]:
# RF
param_grid={
    'n_estimators': [100,1000,1500],
    'max_depth': [5,10,20],
    'min_samples_split': [2,4,6],
    'min_samples_leaf': [2,4,6],
    'max_features':['sqrt'],
    'oob_score': [True]
}

RF=rf(
#    n_estimators=1000,
#    max_depth=10,
#    min_samples_split=2,
#    min_samples_leaf=2,
    max_features='sqrt',
    oob_score=True,
    class_weight='balanced_subsample',
    n_jobs=3,
    random_state=1001
)

model=GridSearchCV(estimator=RF, param_grid=param_grid, cv=5, n_jobs=2, scoring='roc_auc')
model.fit(X_train_ready,y)
print(model.best_params_)
print(model.best_score_)

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100, 'oob_score': True}
0.8793904131826037


In [144]:
# GBM
param_grid={
    'min_samples_split':[10],
    'min_samples_leaf':[3],
    'max_depth':[4],
    'max_features':['sqrt'],
    'subsample': [0.5],
    'n_estimators':[1000],
    'learning_rate':[0.02]
}
clf=gbc(
#    min_samples_split=8,
#    min_samples_leaf=3, 
#    max_depth=4, 
#    max_features='sqrt', 
#    subsample=0.5,
#    n_estimators=1000,
#    learning_rate=0.02,
    random_state=1001
)

model=GridSearchCV(estimator=clf,cv=5,param_grid=param_grid, n_jobs=8, scoring='roc_auc')
model.fit(X_train_ready,y)
print(model.best_params_)
print(model.best_score_)

{'learning_rate': 0.02, 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000, 'subsample': 0.5}
0.8739277619856464


In [139]:
clf=gbc(
    min_samples_split=8,
    min_samples_leaf=3, 
    max_depth=4, 
    max_features='sqrt', 
    subsample=0.5,
    n_estimators=1000,
    learning_rate=0.02,
    random_state=1001,
    
)
clf.fit(X_train_ready,y)
clf.train_score_

array([1.32566093, 1.29774211, 1.28190412, 1.27387905, 1.26778266,
       1.24757806, 1.20318283, 1.22307081, 1.20363014, 1.21546637,
       1.18083239, 1.20609983, 1.16845042, 1.15096262, 1.15322504,
       1.13614775, 1.1291263 , 1.14347369, 1.14246942, 1.10802029,
       1.10273526, 1.09550927, 1.08394781, 1.07648979, 1.04285238,
       1.06580199, 1.06951977, 1.05787638, 1.06065927, 1.01368452,
       1.01967948, 1.01419874, 0.98792856, 1.00941825, 0.99733701,
       0.98973093, 0.97159501, 0.98063376, 0.97130975, 0.98002882,
       0.94927528, 0.93004937, 0.96582841, 0.94048221, 0.94894763,
       0.92695675, 0.92955289, 0.96401313, 0.92090443, 0.91612713,
       0.91204651, 0.90070333, 0.92294257, 0.88448499, 0.86419105,
       0.86536141, 0.84630376, 0.84863063, 0.90200052, 0.87832615,
       0.85697639, 0.88337328, 0.84709607, 0.83669603, 0.86924292,
       0.87126753, 0.87869878, 0.84502826, 0.8490598 , 0.81763491,
       0.86128351, 0.8017726 , 0.87966926, 0.84702102, 0.81838

In [152]:
# XGB
param_grid={
 'learning_rate':[0.02],
 'n_estimators':[1000],
 'max_depth':[4],
 'min_child_weight':[4],
 'gamma':[0],
 'subsample':[0.5],
 'colsample_bytree':[0.5],
 'objective': ['binary:logistic'],
 'nthread':[4],
 'scale_pos_weight':[1],
    'reg_alpha':[0.01],
    'reg_lambda':[0.01]
}
XGBC=xgbc(
 learning_rate =0.002,
 n_estimators=2000,
 max_depth=4,
 min_child_weight=4,
 gamma=0,
 subsample=0.5,
 colsample_bytree=0.5,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=10,
 seed=27,
    reg_alpha=0.01,
    reg_lambda=0.01
)

model=GridSearchCV(estimator=XGBC,cv=5,param_grid=param_grid, n_jobs=8, scoring='roc_auc')
model.fit(X_train_ready,y)
print(model.best_params_)
print(model.best_score_)

{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.02, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 1000, 'nthread': 4, 'objective': 'binary:logistic', 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'scale_pos_weight': 1, 'subsample': 0.5}
0.8676039363621959


In [157]:
# LightGBM
param_grid={
    'num_leaves':[100],
 'learning_rate':[0.02],
 'n_estimators':[1000],
 'max_depth':[4],
 'min_child_weight':[4],
    'min_child_sample':[10],
 'gamma':[0],
 'subsample':[0.5],
 'colsample_bytree':[0.5],
    'reg_alpha':[0.01],
    'reg_lambda':[0.01]
}
LGBMC=lgbmc(
    num_leaves=100,
 learning_rate =0.002,
 n_estimators=2000,
 max_depth=4,
 min_child_weight=4,
    min_child_samples=10,
 gamma=0,
 subsample=0.5,
 colsample_bytree=0.5,
 random_state=27,
    reg_alpha=0.01,
    reg_lambda=0.01,
    class_weight='balanced'
)

model=GridSearchCV(estimator=LGBMC,cv=5,param_grid=param_grid, n_jobs=8, scoring='roc_auc')
model.fit(X_train_ready,y)
print(model.best_params_)
print(model.best_score_)

{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.02, 'max_depth': 4, 'min_child_sample': 10, 'min_child_weight': 4, 'n_estimators': 1000, 'num_leaves': 100, 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'subsample': 0.5}
0.863155187275044


In [159]:
y.value_counts()

0    549
1    340
Name: Survived, dtype: int64

In [163]:
from sklearn.model_selection import StratifiedKFold as SK

In [175]:
skf=SK(n_splits=5, random_state=1001)
skf.get_n_splits(X_train_ready,y)

5

In [176]:
i=0
for x_index, y_index in skf.split(X_train_ready,y):
    i=i+1
    print(i)
    
    print(x_index,y_index)
    print(len(x_index))
    print(len(y_index))
    

1
[167 168 169 170 172 173 174 175 176 177 178 179 180 181 184 187 188 190
 195 196 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213
 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231
 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249
 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285
 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303
 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321
 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339
 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357
 358 359 360 361 362 363 364 365 366 367 368 369 370 371 372 373 374 375
 376 377 378 379 380 381 382 383 384 385 386 387 388 389 390 391 392 393
 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 409 410 411
 412 413 414 415 416 417 418 419 420 421 422 423 

In [171]:
i

5

3.99438202247191